In [1]:
import tensorflow as tf

In [2]:
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data # sample weight can be added here

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses) # additional loss added

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes all the metrics that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

::: tips Valid Layers Only layers exist in both __init__ / build and call are effective ones (saved in checkpoint) :::


### Serving

In [ ]:
path = "tmp"
# some built-in method or @tf.function as serving_default, errors are consumed silently
model.save(path)
tf.saved_model.save(model, path)
tf.keras.models.save_model(model, path)

# errors are thrown out
# model.func is decorated with @tf.function(input_signatures=[])
model.save(path, signatures={'name': model.func})
tf.saved_model.save(model, path, signatures={'name': model.func})
tf.keras.models.save_model(model, path, signatures={'name': model.func})

# errors are thrown out
# model.func is decorated with @tf.function baldly
model.save(path, signatures={'name': model.func.get_concrete_function(some_input_signatures)})
tf.saved_model.save(model, path, signatures={'name': model.func.get_concrete_function(some_input_signatures)})
tf.keras.models.save_model(model, path, signatures={'name': model.func.get_concrete_function(some_input_signatures)})

In [4]:
# original
@tf.function(input_signature=[tf.TensorSpec(shape=[None,None], dtype=tf.float32)])
def add(self, x):
    if x.shape[1] is None:
        print(x)
        return tf.constant('666')
    else:
        return tf.constant(x.shape[1])

# simplified due to an execution with empty tensor
def add(self, x):
    return tf.constant('666')

In [5]:
# Fix one: remove input_signature
@tf.function()
def add(self, x):
    if x.shape[1] is None:
        print(x)
        return tf.constant('666')
    else:
        return tf.constant(x.shape[1])

In [6]:
# Fix two: use tensors in branch conditional
@tf.function(input_signature=[tf.TensorSpec(shape=[None,None], dtype=tf.float32)])
def add(self, x):
    if tf.equal(tf.size(x), 0):  # empty tensor size is 0; [[1,2],[3,4]] size is 4
        print(x)
        return tf.constant('666')
    else:
        return tf.constant(x.shape[1])

In [7]:
# Fix three: run tf functions in eager mode
tf.config.run_functions_eagerly(True)  # tf >= 2.3
tf.config.experimental_run_functions_eagerly(True)  # tf >= 2.0

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [8]:
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32)]
def tf_func(x):
    do_sth(x)
    return

# auto parsed to test(one_arg=, one_arg_1=)
# if multiple tensors are passed through a single argument, they are differentiated by number suffix
@tf.function(input_signatures=[(tf.TensorSpec(shape=None, dtype=tf.int32), tf.TensorSpec(shape=None, dtype=tf.int32))])
def test(one_arg: Tuple(int, int)):
    do_sth(one_arg)
    return

# later use get_concrete_function(tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32))
@tf.function
def bald_tf_func(x):
    do_sth(x)
    return

SyntaxError: invalid syntax (1687953081.py, line 2)

In [9]:
"""Only accept sequence of tensors"""

# one argument
model.signatures['name'](one_arg_in_tensor)
# multiple arguments should be passed as **kwargs
.signatures['name'](**dict_multiple_args)

SyntaxError: invalid syntax (3398124104.py, line 6)

In [10]:
a = tf.constant([[1,2,3],[4,5,6]], tf.int32)
b = tf.constant([1,2], tf.int32)  # one for each dimension

# [[1,2,3,1,2,3],[4,5,6,4,5,6]]
tf.tile(a, b)

2021-10-14 17:33:50.983428: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]], dtype=int32)>

In [12]:
import numpy as np

In [13]:
# 1d indices without batch_dims
ps = np.random.uniform(-1, 1, (5, 2))
ids = [1,0]

# select rows
# (2,) + (2,) => (2, 2)
tf.gather(ps, ids, axis=0)  # r[0] = ps[ids[0]]; r[1] = ps[ids[1]]

# select columns
# (5,) + (2,) => (5, 2)
tf.gather(ps, ids, axis=1)  # r[:, 0] = ps[:, ids[0]]; r[:, 1] = ps[:, ids[1]]


# 2d indices without batch_dims
ps = np.random.uniform(-1, 1, (5, 2))
ids = np.random.randint(0, 2, (5, 3))

# select rows
# (5, 3) + (,2) => (5, 3, 2)
# for i in range(5):
#     for j in range(3):
#         r[i, j, :] = ps[ids[i, j], :]
tf.gather(ps, ids, axis=0)

# select cols
# (5, ) + (5, 3) => (5, 5, 3)
# for i in range(5):
#     for j in range(3):
#         r[:, i, j] = ps[:, ids[i, j]]
tf.gather(ps, ids, axis=1)


# 2d indices with batch_dims
ps = np.random.uniform(-1, 1, (5, 2))
ids = np.random.randint(0, 2, (5, 3))

# batch_dims = 0 takes no effect
tf.gather(ps, ids, axis=1, batch_dims=0)

# if axis = 1
# def manually_batched_gather(ps, ids, axis):
#     batch_dims=1
#     assert batch_dims <= axis
#     assert ids.shape[0] == ps.shape[0]
#     result = []
#     for p,i in zip(ps, ids):
#         r = tf.gather(p, i, axis=axis-batch_dims)
#         result.append(r)
#     return tf.stack(result)
tf.gather(ps, ids, axis=1, batch_dims=1)  # (5, 3)

# 3d indices with batch_dims
ps = np.random.uniform(-1, 1, (3, 5, 2))
ids = np.random.randint(0, 2, (3, 5, 3))

tf.gather(ps, ids, axis=2, batch_dims=1)  # (3, 5, 5, 3)

# if axis = batch_dims,
# for p0, i0 in zip(ps, ids):
#     sub_result = []
#     for p1, i1 in zip(p0, i0):
#         r = tf.gather(p1, i1, axis=0)
#         sub_result.append(r)
#     result.append(tf.stack(sub_result))
# tf.stack(result)
tf.gather(ps, ids, axis=2, batch_dims=2)  # (3, 5, 3)

<tf.Tensor: shape=(3, 5, 3), dtype=float64, numpy=
array([[[ 0.48372874,  0.48372874,  0.83624316],
        [-0.58736895, -0.03821011, -0.03821011],
        [-0.1638027 , -0.1638027 , -0.83502416],
        [ 0.40656184,  0.13559765,  0.40656184],
        [-0.09690313,  0.68085775,  0.68085775]],

       [[-0.45005643, -0.43341404, -0.45005643],
        [ 0.57919562, -0.9923409 ,  0.57919562],
        [ 0.01991931, -0.04308811,  0.01991931],
        [-0.89897645, -0.89897645, -0.89897645],
        [ 0.06809007, -0.57559243, -0.57559243]],

       [[ 0.59922772,  0.59922772,  0.96226704],
        [-0.39183219,  0.44381656,  0.44381656],
        [ 0.27561701, -0.58657842,  0.27561701],
        [-0.38742094, -0.38742094, -0.38742094],
        [-0.40525719, -0.32690152, -0.32690152]]])>

In [14]:
"""
same rank
"""
indices = [[0, 0], [1, 1]]
params = [['a', 'b'], ['c', 'd']]

# ['a', 'd']
tf.gather_nd(params, indices)

indices = [[1], [0]]
params = [['a', 'b'], ['c', 'd']]

# [['c', 'd'], ['a', 'b']]
tf.gather(params, indices)


<tf.Tensor: shape=(2, 1, 2), dtype=string, numpy=
array([[[b'c', b'd']],

       [[b'a', b'b']]], dtype=object)>

In [16]:
"""
batched indices and params
"""
indices = [[1], [0]]  # (2, 1)
params = [
    [['a0', 'b0'], ['c0', 'd0']],
    [['a1', 'b1'], ['c1', 'd1']]
]  # (2, 2, 2)

# [['c0', 'd0'], ['a1', 'b1']]
tf.gather_nd(params, indices, batch_dims = 1)

<tf.Tensor: shape=(2, 2), dtype=string, numpy=
array([[b'c0', b'd0'],
       [b'a1', b'b1']], dtype=object)>

In [18]:
a = tf.constant(
    [
        [1, 1, 1, 4],
        [4, 0, 1, 1],
        [3, 2, 4, 1]
    ]
)

b = tf.constant(
    [
        [3, 1, 3, 4],
        [2, 4, 0, 4],
        [3, 2, 4, 1]
    ]
)

# tf.tanspose([a, b])

In [20]:
tf.print(a)  # tensor value can be printed
tf.print(f'{b}')  # empty tensor value

[[1 1 1 4]
 [4 0 1 1]
 [3 2 4 1]]
[[3 1 3 4]
 [2 4 0 4]
 [3 2 4 1]]


In [22]:
# tf 2.1 eager
from tensorflow.python.framework import random_seed
tf.random.set_seed(0)
random_seed.get_seed(None)  # counter (0, 1654615998)
random_seed.get_seed(None)  # counter (0, 2087043557)

# tf 2.1 graph
from tensorflow.python.framework import random_seed
tf.random.set_seed(0)
random_seed.get_seed(None)  # counter (0, 1)
random_seed.get_seed(None)  # counter (0, 3)

# tf 2.1 different graph
from tensorflow.python.framework import random_seed
tf.random.set_seed(0)
random_seed.get_seed(None)  # counter (0, 2)
random_seed.get_seed(None)  # c

(0, 1806341205)

In [23]:
# since tf.function transform code to graph, we expect it can interact correctly with tf's graph tensors
# however, error occurs if input_signatures exist for model function which requires keras's learning phase(whether training)
# error occurs might due to keras_learning_phase is symbolic(graph) tensor, but func is first a eager function then transformed to graph
@tf.function(input_signatures=[])
model_func():
    depends on learning phase

# Okay, black box success
@tf.function
model_func():
    depends on learning phase

SyntaxError: invalid syntax (2223447619.py, line 5)

In [ ]:
class MaxBlurPooling2D(L.Layer):

    def __init__(self, pool_size: int = 2, kernel_size: int = 3, **kwargs):
        self.pool_size = pool_size
        self.kernel_size = kernel_size

        super(MaxBlurPooling2D, self).__init__(**kwargs)
        
    def get_config(self):
        config = {
            'pool_size': self.pool_size,
            'kernel_size': self.kernel_size
            # in this config should not include the variable from add_weight but only the variables in __init__
        }
        base_config = super(MaxBlurPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
        
    def build(self, input_shape):

        if self.kernel_size == 3:
            bk = tf.constant([[1, 2, 1],
                           [2, 4, 2],
                           [1, 2, 1]])
            bk = bk / tf.math.reduce_sum(bk)
        elif self.kernel_size == 5:
            bk = tf.constant([[1, 4, 6, 4, 1],
                           [4, 16, 24, 16, 4],
                           [6, 24, 36, 24, 6],
                           [4, 16, 24, 16, 4],
                           [1, 4, 6, 4, 1]])
            bk = bk / tf.math.reduce_sum(bk)
        else:
            raise ValueError
        bk = tf.cast(bk,tf.float32)
        bk = tf.repeat(bk, input_shape[3])

        bk = tf.reshape(bk, (self.kernel_size, self.kernel_size, input_shape[3], 1))

        self.blur_kernel = self.add_weight(name='blur_kernel',
                                           shape=(self.kernel_size, self.kernel_size, input_shape[3], 1),
                                           initializer=tf.keras.initializers.constant(bk),
                                           trainable=False)

        super(MaxBlurPooling2D, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):

        x = tf.nn.pool(x, (self.pool_size, self.pool_size),
                       strides=(1, 1), padding='SAME', pooling_type='MAX', data_format='NHWC')
        x = K.backend.depthwise_conv2d(x, self.blur_kernel, padding='same', strides=(self.pool_size, self.pool_size))

        return x

    def compute_output_shape(self, input_shape):
        return input_shape[0], int(tf.math.ceil(input_shape[1] / 2)), int(tf.math.ceil(input_shape[2] / 2)), input_shape[3]